# UV-Vis Gaussian Input Generation and Export

This notebook streamlines the process of generating Gaussian input files for a set of molecules represented by SMILES strings, typically from a UV-Vis dataset. It:

- Loads and processes a CSV of molecular SMILES and associated spectral data.
- Converts each SMILES string into a 3D-optimized geometry using Open Babel.
- Writes multi-step Gaussian input files, including:
  - HF or DFT geometry optimizations and frequency calculations.
  - TDDFT single-point calculations with user-defined methods and basis sets.
- Organizes all `.com` files in a designated output directory.
- Optionally compresses the directory into a `.zip` file for cluster upload (e.g., to Curie).

This pipeline is ideal for high-throughput quantum chemistry input preparation for excited-state calculations.


### Download Modules

In [ ]:
%%capture
!pip install PubChemPy
!pip install pandas
!pip install openbabel

In [ ]:
import pandas as pd
from openbabel import pybel


### Download UV-Vis data

In [ ]:
! wget https://raw.githubusercontent.com/PNNL-CompBio/ML_UVvisModels/main/Data/UV_w_SMILES.csv

### Load data in Pandas

In [ ]:
import pandas as pd

In [ ]:
# Define the file path to the CSV file containing UV-Vis data with SMILES strings
uvvis_file = "UV_w_SMILES.csv"

# Read the CSV file into a pandas DataFrame
uvvis_data = pd.read_csv(uvvis_file,header=None)

# Display the contents of the DataFrame
uvvis_data

### Convert SMILES to Skeletal Structure

In [ ]:
for i in range(932):
    smi_string = uvvis_data.iloc[i, 0]

In [ ]:
from rdkit import Chem

# Convert the SMILES string at index 690 in the 'uvvis_data' DataFrame to an RDKit molecule object
m = Chem.MolFromSmiles(uvvis_data.iloc[35, 0])

# Display the RDKit molecule object
print(uvvis_data.iloc[35,0])

### Convert SMILES to XYZ

[Pybel](https://open-babel.readthedocs.io/en/latest/UseTheLibrary/Python_Pybel.html) is a Python wrapper for Open Babel, a chemical toolbox designed to speak the many languages of chemical data. It simplifies the process of converting between different chemical file formats and manipulating chemical data.

For more information about Pybel and Open Babel, visit the [Open Babel website](https://openbabel.org/index.html).

In [ ]:
from openbabel import pybel


# Function to convert SMILES to XYZ
def smiles_to_xyz(smiles_string):
    # Convert SMILES to molecule
    mol = pybel.readstring("smi", smiles_string)
    
    # Add hydrogens
    mol.addh()
    
    # Generate 3D coordinates
    mol.make3D()
    
    # Optimize geometry
    mol.localopt()
    
    # Get XYZ coordinates
    xyz = mol.write("xyz")

    return xyz

This cell demonstrates the conversion of a SMILES string to XYZ coordinates using the `smiles_to_xyz` function. The example uses the SMILES string for cyclohexane ("C1CCCCC1"). The function `smiles_to_xyz` takes this SMILES string and converts it into the corresponding XYZ coordinates, which are stored in the variable `cyclohexane_coordinates`.


In [ ]:
# Example SMILES string
string = uvvis_data.iloc[35, 0]

# Convert to XYZ
coordinates = smiles_to_xyz(string)

### Print Gaussian Files

#### Gaussian Coordinate Print Function

This function automates the creation of a multi-step Gaussian input file from a SMILES string. It performs:

1. **Step 0**: 3D-coordinate generation via Open Babel and a HF optimization + frequency calculation.
2. **Step 1...N**: Subsequent optimizations or TDDFT calculations at user-specified levels of theory/basis.

**Parameters:**
- `smiles` (`str`): SMILES representation of the molecule.
- `filename_prefix` (`str`): Prefix for all output files (e.g., `molecule_0000`).
- `path` (`str`): Pathway to folder (e.g., `gauss_files`).
- `methods_basis` (`List[Tuple[str,str]]`): Sequence of `(method, basis)` pairs defining each calculation step. The last step may be a TDDFT run.
- `dispersion` (`str`, optional): Dispersion correction keyword (e.g., `GD3`, `GD3BJ`). Omit for no dispersion.

**Behavior:**
- Calls `smiles_to_xyz` to generate and optimize 3D geometry, then writes an XYZ file.
- Writes a Gaussian input file (`.com`) in `path`, with Link1 sections for each step:
  - `opt freq` for optimization + frequency.
  - `TD=(nstates=20,50-50)` for the final TDDFT.
- Uses fixed directives `%chk`, `%mem=1000mb`, `%nprocshared=64`.

**Output:**
- Gaussian input: `gauss_files2/{filename_prefix}.com`.

In [ ]:
# Main function to generate Gaussian input files
def smiles_to_gaussian(smiles, filename_prefix, path, methods_basis, dispersion=None):
    # Step 1: Create XYZ
    xyz_str = smiles_to_xyz(smiles)
    lines = xyz_str.split("\n")
    
    # Gaussian Input
    gauss_path = f"{path}/{filename_prefix}.com"
    with open(gauss_path, "w") as gauss_file:
        chk = f"{filename_prefix}.chk"
        mem = "1000mb"
        nproc = "64"

        # Write initial step with coordinates
        gauss_file.write(f"%chk={chk}\n%mem={mem}\n%nprocshared={nproc}\n")
        disp_str = f" EmpiricalDispersion={dispersion}" if dispersion else ""
        method0, basis0 = methods_basis[0]
        gauss_file.write(f"# opt freq {method0}/{basis0}\n\n")
        gauss_file.write(f"{smiles} -- Step 0: Optimization + Frequency ({method0}/{basis0})\n\n0 1\n")
        gauss_file.write("\n".join(lines[2:]) + "\n\n")

        # Link1 steps
        numsteps = len(methods_basis)
        for step, (method, basis) in enumerate(methods_basis[1:], start=1):
            gauss_file.write("--Link1--\n")
            gauss_file.write(f"%chk={chk}\n%mem={mem}\n%nprocshared={nproc}\n")
            if step == numsteps-1:
                gauss_file.write(f"# TD=(nstates=20,50-50) {method}/{basis}{disp_str} geom=check guess=read\n\n")
                gauss_file.write(f"Step {step}: TDDFT ({method}/{basis})\n\n")
            else:
                gauss_file.write(f"# opt freq {method}/{basis}{disp_str} geom=check guess=read\n\n")
                gauss_file.write(f"Step {step}: Optimization + Frequency ({method}/{basis})\n\n")
            gauss_file.write("0 1\n\n")

    print(f"Generated {gauss_path}")

### A Simple Example 

In [ ]:
# Example Usage:
methods_basis = [
    ("HF", "cc-pVDZ"),
    ("HF", "cc-pVDZ")
]

import os 
path = "gauss_files_simple"
# Ensure directory exist
os.makedirs("gauss_files_simple", exist_ok=True)


smiles_to_gaussian("C1=CC=CC=C1", "molecule_0000", path, methods_basis)

#### Batch Generation of Gaussian Inputs

This snippet captures output and loops over all entries in `uvvis_data` to generate Gaussian input files in bulk:

- **`methods_basis`**: List of `(method, basis)` tuples defining the calculation steps for each molecule.
- **`path`**: Directory where the `.com` files are stored (e.g., `gauss_files_simple`).
- **Loop**: Iterates over an index range to process multiple SMILES entries.

In [ ]:
%%capture

methods_basis = [
    ("HF", "cc-pVDZ"),
    ("HF", "cc-pVDZ"),
]

path = "gauss_files_simple"

for i in range(932):
    smi_string = uvvis_data.iloc[i, 0]
    prefix = f"molecule_{i:04d}"
    smiles_to_gaussian(smi_string, prefix, path, methods_basis)


#### Compress Gaussian Path Folder

Once input files are generated, you can zip the entire folder. This compresses the contents of the directory specified by `path`. Make sure that `path` is defined above.


In [ ]:
!zip -r {path}.zip {path}

Now just download your gaussian path and upload it to Curie. You're done!